In [ ]:
#Preferrably use High-RAM on colab, os.cpu_count() >= 8
import os
import subprocess
from google.colab import drive
drive.mount('/content/drive')
%pip install Bio
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import shutil

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.6 MB/s eta 0:00:00


# Running ***Cawligner***

In [ ]:
# Install dependencies
!apt-get update -qq
!apt-get install -y cmake g++ make git

# Clone the repo
!git clone https:/github.com/veg/cawlign.git
%cd cawlign

# Build cawlign
!cmake ./
!make

# Install (optional, otherwise you can just call ./cawlign directly)
!make install


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
g++ set to manually installed.
make is already the newest version (4.3-4.1build1).
make set to manually installed.
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
git is already the newest version (1:2.34.1-1ubuntu1.15).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
Cloning into 'cawlign'...
remote: Enumerating objects: 576, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 576 (delta 21), reused 32 (delta 12), pack-reused 522 (from 1)
Receiving objects: 100% (576/576), 2.94 MiB | 14.29 MiB/s, done.
Resolving deltas: 100% (269/269), done.
/content/cawlign

In [ ]:
#matrix = """4 -1 -2 -2 0 -1 -1 0 -2 -1 -1 -1 -1 -2 -1 1 0 -3 -2 0 -2 -1 -1 -1 -4 -1 5 0 -2 -3 1 0 -2 0 -3 -2 2 -1 -3 -2 -1 -1 -3 -2 -3 -1 -2 0 -1 -4 -2 0 6 1 -3 0 0 0 1 -3 -3 0 -2 -3 -2 1 0 -4 -2 -3 4 -3 0 -1 -4 -2 -2 1 6 -3 0 2 -1 -1 -3 -4 -1 -3 -3 -1 0 -1 -4 -3 -3 4 -3 1 -1 -4 0 -3 -3 -3 9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1 -3 -1 -3 -1 -4 -1 1 0 0 -3 5 2 -2 0 -3 -2 1 0 -3 -1 0 -1 -2 -1 -2 0 -2 4 -1 -4 -1 0 0 2 -4 2 5 -2 0 -3 -3 1 -2 -3 -1 0 -1 -3 -2 -2 1 -3 4 -1 -4 0 -2 0 -1 -3 -2 -2 6 -2 -4 -4 -2 -3 -3 -2 0 -2 -2 -3 -3 -1 -4 -2 -1 -4 -2 0 1 -1 -3 0 0 -2 8 -3 -3 -1 -2 -1 -2 -1 -2 -2 2 -3 0 -3 0 -1 -4 -1 -3 -3 -3 -1 -3 -3 -4 -3 4 2 -3 1 0 -3 -2 -1 -3 -1 3 -3 3 -3 -1 -4 -1 -2 -3 -4 -1 -2 -3 -4 -3 2 4 -2 2 0 -3 -2 -1 -2 -1 1 -4 3 -3 -1 -4 -1 2 0 -1 -3 1 1 -2 -1 -3 -2 5 -1 -3 -1 0 -1 -3 -2 -2 0 -3 1 -1 -4 -1 -1 -2 -3 -1 0 -2 -3 -2 1 2 -1 5 0 -2 -1 -1 -1 -1 1 -3 2 -1 -1 -4 -2 -3 -3 -3 -2 -3 -3 -3 -1 0 0 -3 0 6 -4 -2 -2 1 3 -1 -3 0 -3 -1 -4 -1 -2 -2 -1 -3 -1 -1 -2 -2 -3 -3 -1 -2 -4 7 -1 -1 -4 -3 -2 -2 -3 -1 -1 -4 1 -1 1 0 -1 0 0 0 -1 -2 -2 0 -1 -2 -1 4 1 -3 -2 -2 0 -2 0 -1 -4 0 -1 0 -1 -1 -1 -1 -2 -2 -1 -1 -1 -1 -2 -1 1 5 -2 -2 0 -1 -1 -1 -1 -4 -3 -3 -4 -4 -2 -2 -3 -2 -2 -3 -2 -3 -1 1 -4 -3 -2 11 2 -3 -4 -2 -2 -1 -4 -2 -2 -2 -3 -2 -1 -2 -3 2 -1 -1 -2 -1 3 -3 -2 -2 2 7 -1 -3 -1 -2 -1 -4 0 -3 -3 -3 -1 -2 -2 -3 -3 3 1 -2 1 -1 -2 -2 0 -3 -1 4 -3 2 -2 -1 -4 -2 -1 4 4 -3 0 1 -1 0 -3 -4 0 -3 -3 -2 0 -1 -4 -3 -3 4 -3 0 -1 -4 -1 -2 -3 -3 -1 -2 -3 -4 -3 3 3 -3 2 0 -3 -2 -1 -2 -1 2 -3 3 -3 -1 -4 -1 0 0 1 -3 4 4 -2 0 -3 -3 1 -1 -3 -1 0 -1 -2 -2 -2 0 -3 4 -1 -4 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 1"""
# Split into numbers
#nums = matrix.split()

# Convert to string with commas
#flattened = ",".join(nums)

# Add prefix
#flattened = "cost =" + flattened

#print(flattened)

In [ ]:
#Sample cawlign command
#! /content/cawlign/cawlign \
#  -o "/content/drive/MyDrive/Paper_Results/Output_files/HA_aligned.fasta" \
#  -s "/content/cawlign/res/scoring/BLOSUM62_copy" \
#  -r "/content/drive/MyDrive/Paper_Results/Reference_files/HA_ref.fasta" \
#  -t protein \
#  "/content/drive/MyDrive/Paper_Results/Input_files/Compiled_human_HA.fasta"

     100 sequences     200 sequences     300 sequences     400 sequences     500 sequences     600 sequences     700 sequences     800 sequences     900 sequences    1000 sequences    1100 sequences    1200 sequences    1300 sequences    1400 sequences    1500 sequences    1600 sequences

In [ ]:
ls

drive/  sample_data/


In [ ]:
reference_dir = "/content/drive/MyDrive/Paper_Results/Reference_files/NT Files"
def remove_last_three_bases(reference_dir):
    # Loop through all fasta files in the directory
    for file in os.listdir(reference_dir):
        if file.endswith(".fasta"):
            file_path = os.path.join(reference_dir, file)

            record = SeqIO.read(file_path, "fasta")

            record.seq = record.seq[:-3] if len(record.seq) >= 3 else record.seq

            SeqIO.write(record, file_path, "fasta")
            print(f"Trimmed stop codon in place: {file}")

remove_last_three_bases(reference_dir)

Trimmed stop codon in place: HA_ref.fasta
Trimmed stop codon in place: NS1_ref.fasta
Trimmed stop codon in place: NS2_ref.fasta
Trimmed stop codon in place: M1_ref.fasta
Trimmed stop codon in place: M2_ref.fasta
Trimmed stop codon in place: NA_ref.fasta
Trimmed stop codon in place: NP_ref.fasta
Trimmed stop codon in place: PA_ref.fasta
Trimmed stop codon in place: PB1_ref.fasta
Trimmed stop codon in place: PB2_ref.fasta


In [ ]:
# Paths
cawlign_exec = "cawlign"

scoring_matrix = "/content/cawlign/res/scoring/BLOSUM62_reso_copy"


input_dir = "/content/drive/MyDrive/Paper_Results/Input_files_Avian_All_NT"
output_dir = "/content/drive/MyDrive/Paper_Results/Cawligner_output_files/Avian"

#input_dir = "/content/drive/MyDrive/Paper_Results/Input_files_Mammal_All_NT"
#output_dir = "/content/drive/MyDrive/Paper_Results/Cawligner_output_files/Mammal"

#After Removing the stop codon
reference_dir = "/content/drive/MyDrive/Paper_Results/Reference_files/NT Files"


# Ensure output folder exists
os.makedirs(output_dir, exist_ok=True)

# Loop through all fasta files in input_dir
for file in os.listdir(reference_dir):
    if file.endswith(".fasta"):
        #input_path = os.path.join(input_dir, file)

        #print(file)
        # Skip specific file (DO NOT REMOVE)
        #if file == "Compiled_human_HA.fasta":
        #    continue

        # Extract gene name (assumes filename has _GENE.fasta or similar)
        #gene = file.split("_")[-1].replace(".fasta", "") #For when looping through the input directory
        gene = file.split("_")[0]
        print("Gene:", gene)

        if gene == "NS1" or gene == "NS2":
          input_path = os.path.join(input_dir, f"Compiled_NS.fasta")
        elif gene == "M1" or gene == "M2":
          input_path = os.path.join(input_dir, f"Compiled_M.fasta")
        else:
          input_path = os.path.join(input_dir, f"Compiled_{gene}.fasta")

        output_path = os.path.join(output_dir, f"{gene}_aligned.fasta")
        reference_path = os.path.join(reference_dir, f"{gene}_ref.fasta")

        # Check if reference exists
        if not os.path.exists(reference_path):
            print(f"Skipping {gene} — reference file not found: {reference_path}")
            continue

        # Build the command
        cmd = [
            cawlign_exec,
            "-r", reference_path,
            "-o", output_path,
            "-s", scoring_matrix,
            "-t", "codon",
            input_path
        ]

        print(f"Running cawlign for gene: {gene}")

        try:
          result = subprocess.run(
              cmd,
              check=True,
              stdout=subprocess.PIPE,
              stderr=subprocess.PIPE,
              text=True  # returns strings instead of bytes
          )
          print("✅ cawlign output:")
          print(result.stdout)

        except subprocess.CalledProcessError as e:
            print("❌ cawlign failed!")
            print("Command:", e.cmd)
            print("Return code:", e.returncode)
            print("Stdout:\n", e.stdout)
            print("Stderr:\n", e.stderr)

print("Done")


Gene is: HA
Running cawlign for gene: HA
✅ cawlign output:

Gene is: NS1
Running cawlign for gene: NS1
✅ cawlign output:

Gene is: NS2
Running cawlign for gene: NS2
✅ cawlign output:

Gene is: M1
Running cawlign for gene: M1
✅ cawlign output:

Gene is: M2
Running cawlign for gene: M2
✅ cawlign output:

Gene is: NA
Running cawlign for gene: NA
✅ cawlign output:

Gene is: NP
Running cawlign for gene: NP
✅ cawlign output:

Gene is: PA
Running cawlign for gene: PA
✅ cawlign output:

Gene is: PB1
Running cawlign for gene: PB1
✅ cawlign output:

Gene is: PB2
Running cawlign for gene: PB2
✅ cawlign output:

Done


# Running the ***tn93-cluster***

In [ ]:
%cd ..

CMakeCache.txt       fasta_diff*           README.md     ShortestPathTN93*
CMakeFiles/          fasta_digest*         readreduce*   src/
cmake_install.cmake  install_manifest.txt  scripts/      tn93*
CMakeLists.txt       LICENSE               selectreads*  tn93-cluster*
data/                Makefile              seqcoverage*  validate_fasta*
Dockerfile           nucfreqsfasta*        seqdiff*      VERSION.txt


In [ ]:
!git clone https:/github.com/veg/tn93.git
%cd tn93
!cmake ./
!make install

Cloning into 'tn93'...
remote: Enumerating objects: 928, done.
remote: Counting objects: 100% (290/290), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 928 (delta 164), reused 240 (delta 135), pack-reused 638 (from 1)
Receiving objects: 100% (928/928), 315.56 KiB | 3.19 MiB/s, done.
Resolving deltas: 100% (557/557), done.
/content/tn93
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting

In [ ]:
# === PATHS ===
tn93_exec = "tn93-cluster"   # <-- update this

species = "Mammal"
print(species)
prior_clade_dir = "/content/drive/MyDrive/Paper_Results/Input_files_"+species+"_Prior_Clades_Compiled_NT"
filtered_nt_dir = "/content/drive/MyDrive/Paper_Results/Cawligner_output_files/"+species+"/Prior_clades_filtered"
aligned_nt_dir = "/content/drive/MyDrive/Paper_Results/Cawligner_output_files/"+species
tn93_output_dir = "/content/drive/MyDrive/Paper_Results/TN93_results/"+species
current_nt_dir = "/content/drive/MyDrive/Paper_Results/Cawligner_output_files/" + species + "/Current_clades_filtered"
print(prior_clade_dir)
# tn93-cluster parameters
tn93_params = ["-l", "500", "-t", "0.005", "-f"]

# Ensure output folders exist
os.makedirs(filtered_nt_dir, exist_ok=True)
os.makedirs(aligned_nt_dir, exist_ok=True)
os.makedirs(tn93_output_dir, exist_ok=True)
os.makedirs(current_nt_dir, exist_ok=True)

def seq_filter(aligned_nt_fasta, prior_clade_fasta, filtered_fasta_path, current_fasta_path):
    """
    Select sequences from aligned_nt_fasta that are present in prior_clade_fasta
    and save them in filtered_fasta_path.
    Save the current ones in the current clade at current_fasta_path.
    """

    # Load raw NT sequences into a dictionary
    nt_dict = {rec.id: rec.seq for rec in SeqIO.parse(prior_clade_fasta, "fasta")}

    filtered_nt_records = []
    skipped_nt_records = []
    count_skips = 0
    count_kept = 0

    for aa_record in SeqIO.parse(aligned_nt_fasta, "fasta"):
        if aa_record.id not in nt_dict:
            skipped_nt_records.append(aa_record)
            count_skips += 1
        else:
            filtered_nt_records.append(aa_record)
            count_kept += 1

    # Save both
    #SeqIO.write(filtered_nt_records, filtered_fasta_path, "fasta")
    SeqIO.write(skipped_nt_records, current_fasta_path, "fasta")

    print(f"✅ Done: Prior Clade {count_kept}, Current Clade {count_skips}, total in compiled file: {len(list(SeqIO.parse(aligned_nt_fasta, "fasta")))}")
    return filtered_nt_records, skipped_nt_records


# === MAIN PIPELINE ===
for file in os.listdir(aligned_nt_dir):
    if file.endswith(".fasta"):
        aligned_fasta_path = os.path.join(aligned_nt_dir, file)
        gene = file.split("_")[0]
        #if gene != "HA":  # <-- optional filter
        #    continue
        print(gene)

        if gene == "NS1" or gene == "NS2":
            prior_clade_path = os.path.join(prior_clade_dir, f"Compiled_prior_NS.fasta")
        elif gene == "M1" or gene == "M2":
            prior_clade_path = os.path.join(prior_clade_dir, f"Compiled_prior_M.fasta")
        else:
            prior_clade_path = os.path.join(prior_clade_dir, f"Compiled_prior_{gene}.fasta")

        filtered_fasta_path = os.path.join(filtered_nt_dir, f"Filtered_aligned_{gene}.fasta")
        current_fasta_path = os.path.join(current_nt_dir, f"Filtered_current_{gene}.fasta")

        # Step 1: filter & save skipped ones
        filtered_nt_records, skipped_nt_records = seq_filter(aligned_fasta_path, prior_clade_path,
                                                             filtered_fasta_path, current_fasta_path)


        continue
        # Step 2: run tn93-cluster on the filtered ones
        tn93_output_path = os.path.join(tn93_output_dir, f"{gene}_tn93.fasta")
        cmd = [tn93_exec] + tn93_params + [filtered_fasta_path]

        print(f"🚀 Running tn93-cluster for {gene}")
        try:
            result = subprocess.run(
                cmd,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True,
                check=True
            )
            with open(tn93_output_path, "w") as out_f:
                out_f.write(result.stdout)

            print(f"✅ {tn93_exec} finished successfully")

        except subprocess.CalledProcessError as e:
            print(f"❌ {tn93_exec} failed!")
            print("Command:", e.cmd)
            print("Return code:", e.returncode)
            print("Stdout:\n", e.stdout)
            print("Stderr:\n", e.stderr)
        print()
print("Done")

Mammal
/content/drive/MyDrive/Kuchipudi Lab/Paper_Results/Input_files_Mammal_Prior_Clades_Compiled_NT
HA
✅ Done: Prior Clade 281, Current Clade 4471, total in compiled file: 4752
NS1
✅ Done: Prior Clade 281, Current Clade 4471, total in compiled file: 4752
NS2
✅ Done: Prior Clade 281, Current Clade 4471, total in compiled file: 4752
M1
✅ Done: Prior Clade 281, Current Clade 4471, total in compiled file: 4752
M2
✅ Done: Prior Clade 281, Current Clade 4471, total in compiled file: 4752
NA
✅ Done: Prior Clade 281, Current Clade 4469, total in compiled file: 4750
NP
✅ Done: Prior Clade 280, Current Clade 4471, total in compiled file: 4751
PA
✅ Done: Prior Clade 281, Current Clade 4469, total in compiled file: 4750
PB1
✅ Done: Prior Clade 281, Current Clade 4472, total in compiled file: 4753
PB2
✅ Done: Prior Clade 281, Current Clade 4471, total in compiled file: 4752
Done


# Running ***HyPhy cln***

In [ ]:
%ls

drive/  sample_data/


In [ ]:
%cd ..

/content


In [ ]:
!git clone --branch 2.5.83 https:/github.com/veg/hyphy.git
%cd hyphy
!cmake ./
!make -j $(nproc)
!make install

Cloning into 'hyphy'...
remote: Enumerating objects: 36914, done.
remote: Counting objects: 100% (937/937), done.
remote: Compressing objects: 100% (315/315), done.
remote: Total 36914 (delta 683), reused 655 (delta 615), pack-reused 35977 (from 4)
Receiving objects: 100% (36914/36914), 122.18 MiB | 22.73 MiB/s, done.
Resolving deltas: 100% (26937/26937), done.
Note: switching to 'f34a1e0335afd67dbd41a5ab38489ffda031a526'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

/content/hyphy
-- The CXX compiler identification is GNU

In [ ]:
!hyphy --help

usage: hyphy or HYPHYMPI [-h] [--help][-c] [-d] [-i] [-p] [BASEPATH=directory path] [CPU=integer] [LIBPATH=library path] [USEPATH=library path] [<standard analysis name> or <path to hyphy batch file>] [--keyword value ...] [positional arguments ...]

Execute a HyPhy analysis, either interactively, or in batch mode
optional flags:
  -h --help                show this help message and exit
  -c                       calculator mode; causes HyPhy to drop into an expression evaluation until 'exit' is typed
  -d                       debug mode; causes HyPhy to drop into an expression evaluation mode upon script error
  -i                       interactive mode; causes HyPhy to always prompt the user for analysis options, even when defaults are available
  -m                       write diagnostic messages to messages.log
optional global arguments:
  BASEPATH=directory path  defines the base directory for all path operations (default is pwd)
  CPU=integer              if compiled with OpenM

In [ ]:

species = "Avian"  # or "Mammal"
print("Species:", species)
current_nt_dir = f"/content/hyphy/{species}/Current_clades_filtered"
output_dir = f"/content/hyphy/cln_results/{species}"

# make sure output dir exists
os.makedirs(output_dir, exist_ok=True)

for file in os.listdir(current_nt_dir):
    if file.endswith(".fasta"):  # only fasta files
        input_file = os.path.join(current_nt_dir, file)
        #gene = file.split("_")[0] #Filtered_current_HA.fasta

        output_file = os.path.join(output_dir, file.replace(".fasta", "_cln.fasta"))

        print(f"Running hyphy cln on {file} ...")

        cmd = [
            "hyphy", "cln"
        ]
        proc = subprocess.run(
            cmd,
            input=f"1\n{input_file}\n3\n{output_file}\n",
            text=True,
            capture_output=True
        )

        if proc.returncode != 0:
            print(f"❌ Error on {file}: {proc.stderr}")
        else:
            print(f"✅ Finished {file}, output -> {output_file}")


# ***Combining Files***

In [ ]:
import os
import json
from io import StringIO
from Bio import SeqIO

species = ["Mammal", "Avian"]

for folder in species:
    tn93_path = f"/content/drive/MyDrive/Paper_Results/TN93_results/{folder}"
    cln_path = f"/content/drive/MyDrive/Paper_Results/cln_results/{folder}"
    iq_tree_input_path = f"/content/drive/MyDrive/Paper_Results/IQ_Tree_Input/{folder}"

    os.makedirs(iq_tree_input_path, exist_ok=True)

    for cln_file in os.listdir(cln_path):
        if cln_file.endswith(".fasta"):
            gene = cln_file.split("_")[2]
            print(f"\nGene: {gene}")

            cln_file_path = os.path.join(cln_path, cln_file)
            tn93_file_path = os.path.join(tn93_path, gene + "_tn93.fasta")

            # --- parse CLN fasta
            cln_fasta = list(SeqIO.parse(cln_file_path, "fasta"))

            # --- parse TN93 (JSON stored in fasta-like file)
            with open(tn93_file_path) as f:
                data = json.load(f)

            fasta_str = "\n".join(entry["centroid"].strip() for entry in data)
            tn93_fasta = list(SeqIO.parse(StringIO(fasta_str), "fasta"))

            print(
                f"CLN: {len(cln_fasta)} | "
                f"TN93: {len(tn93_fasta)}"
            )

            # --- combine CLN and TN93
            combined_records = cln_fasta + tn93_fasta

            # --- write output
            out_file = os.path.join(iq_tree_input_path, gene + "_combined.fasta")
            SeqIO.write(combined_records, out_file, "fasta")

            print(f"✅ {gene}: wrote {len(combined_records)} seqs -> {out_file}")



=== NP ===
CLN: 1194 | TN93: 144
✅ NP: wrote 1338 seqs -> /content/drive/MyDrive/Kuchipudi Lab/Paper_Results/IQ_Tree_Input/Mammal/NP_combined.fasta

=== NA ===
CLN: 1353 | TN93: 160
✅ NA: wrote 1513 seqs -> /content/drive/MyDrive/Kuchipudi Lab/Paper_Results/IQ_Tree_Input/Mammal/NA_combined.fasta

=== M2 ===
CLN: 240 | TN93: 141
✅ M2: wrote 381 seqs -> /content/drive/MyDrive/Kuchipudi Lab/Paper_Results/IQ_Tree_Input/Mammal/M2_combined.fasta

=== M1 ===
CLN: 591 | TN93: 126
✅ M1: wrote 717 seqs -> /content/drive/MyDrive/Kuchipudi Lab/Paper_Results/IQ_Tree_Input/Mammal/M1_combined.fasta

=== NS1 ===
CLN: 791 | TN93: 134
✅ NS1: wrote 925 seqs -> /content/drive/MyDrive/Kuchipudi Lab/Paper_Results/IQ_Tree_Input/Mammal/NS1_combined.fasta

=== HA ===
CLN: 1579 | TN93: 161
✅ HA: wrote 1740 seqs -> /content/drive/MyDrive/Kuchipudi Lab/Paper_Results/IQ_Tree_Input/Mammal/HA_combined.fasta

=== PA ===
CLN: 1839 | TN93: 145
✅ PA: wrote 1984 seqs -> /content/drive/MyDrive/Kuchipudi Lab/Paper_Results

In [ ]:
##
folders = ["Avian", "Mammal"]

base_in = "/content/drive/MyDrive/Paper_Results/IQ_Tree_Input/"
base_filtered = "/content/drive/MyDrive/Paper_Results/cln_results/"
base_out = "/content/drive/MyDrive/Paper_Results/IQ_Tree_Input/"  # save new files here
os.makedirs(base_out, exist_ok=True)

for folder in folders:
    aln_dir = os.path.join(base_in, folder)
    filt_dir = os.path.join(base_filtered, folder)
    out_dir = os.path.join(base_out, folder)
    os.makedirs(out_dir, exist_ok=True)

    # find all combined FASTAs in alignment input
    for fname in os.listdir(aln_dir):
        if fname.endswith("_combined.fasta"):
            aln_path = os.path.join(aln_dir, fname)

            # guess which filtered file to use (you can adjust this if needed)
            gene = fname.replace("_combined.fasta", "")
            filt_fname = f"Filtered_current_{gene}_cln.fasta"
            filt_path = os.path.join(filt_dir, filt_fname)

            if not os.path.exists(filt_path):
                print(f"⚠️ No filtered file for {folder}/{gene}, skipping")
                continue

            # collect IDs from filtered FASTA
            filtered_ids = {rec.id for rec in SeqIO.parse(filt_path, "fasta")}

            # update IDs in alignment FASTA if they are in the filtered set
            updated_records = []
            for rec in SeqIO.parse(aln_path, "fasta"):
                if rec.id in filtered_ids:
                    rec.id = rec.id + "_2.3.3.4b"
                    rec.description = rec.id  # keep description in sync
                updated_records.append(rec)

            # save updated alignment FASTA
            out_path = os.path.join(out_dir, fname)
            SeqIO.write(updated_records, out_path, "fasta")
            print(f"✅ {folder}/{gene}: saved labeled FASTA -> {out_path}")



✅ Avian/NP: saved labeled FASTA -> /content/drive/MyDrive/Kuchipudi Lab/Paper_Results/IQ_Tree_Input/Avian/NP_combined.fasta
✅ Avian/NA: saved labeled FASTA -> /content/drive/MyDrive/Kuchipudi Lab/Paper_Results/IQ_Tree_Input/Avian/NA_combined.fasta
✅ Avian/M2: saved labeled FASTA -> /content/drive/MyDrive/Kuchipudi Lab/Paper_Results/IQ_Tree_Input/Avian/M2_combined.fasta
✅ Avian/M1: saved labeled FASTA -> /content/drive/MyDrive/Kuchipudi Lab/Paper_Results/IQ_Tree_Input/Avian/M1_combined.fasta
✅ Avian/NS1: saved labeled FASTA -> /content/drive/MyDrive/Kuchipudi Lab/Paper_Results/IQ_Tree_Input/Avian/NS1_combined.fasta
✅ Avian/HA: saved labeled FASTA -> /content/drive/MyDrive/Kuchipudi Lab/Paper_Results/IQ_Tree_Input/Avian/HA_combined.fasta
✅ Avian/PA: saved labeled FASTA -> /content/drive/MyDrive/Kuchipudi Lab/Paper_Results/IQ_Tree_Input/Avian/PA_combined.fasta
✅ Avian/PB1: saved labeled FASTA -> /content/drive/MyDrive/Kuchipudi Lab/Paper_Results/IQ_Tree_Input/Avian/PB1_combined.fasta
✅ Av

In [ ]:
import os
from Bio import SeqIO

# Input and output directories (adjust if needed)
input_base = "/content/hyphy/IQ_Tree_Input"
output_base = "/content/hyphy/IQ_Tree_Input"

os.makedirs(output_base, exist_ok=True)

for folder in ["Avian", "Mammal"]:
    in_dir = os.path.join(input_base, folder)
    out_dir = os.path.join(output_base, folder)
    os.makedirs(out_dir, exist_ok=True)

    for fname in os.listdir(in_dir):
        if fname.endswith("_combined.fasta"):
            in_path = os.path.join(in_dir, fname)
            out_path = os.path.join(out_dir, fname)

            new_records = []
            for rec in SeqIO.parse(in_path, "fasta"):
                # Rename only if "_2.3.3.4" is present
                if rec.id.endswith("_2.3.3.4"):
                    rec.id = rec.id.replace("_2.3.3.4", "_2.3.3.4b")
                new_records.append(rec)

            SeqIO.write(new_records, out_path, "fasta")
            print(f"✅ Updated labels in {folder}/{fname} -> {out_path}")


# Running ***IQTree 2***

In [ ]:
%ls

sample_data/


In [ ]:

!cp -r "/content/drive/MyDrive/Paper_Results/IQ_Tree_Input" "/content/hyphy/"

cp: cannot stat '/content/drive/MyDrive/Paper_Results/IQ_Tree_Input': No such file or directory


In [ ]:
%cd ..

/


In [ ]:
# Download IQ-TREE 2.4.0 for Linux (Intel)

!wget -q https:/github.com/iqtree/iqtree2/releases/download/v2.4.0/iqtree-2.4.0-Linux-intel.tar.gz -O iqtree2.tgz

# Extract
!tar -xzf iqtree2.tgz

# Move binary
!mv iqtree-2.4.0-Linux-intel/bin/iqtree2 /usr/local/bin/

# Make it executable
!chmod +x /usr/local/bin/iqtree2


gzip: stdin: unexpected end of file
tar: Child returned status 1
tar: Error is not recoverable: exiting now


In [ ]:
%cd hyphy/

In [ ]:
import os
import subprocess
import time

species = ["Mammal", "Avian"]
base_out = "/content/hyphy/IQ_Tree_output"
os.makedirs(base_out, exist_ok=True)

# Detect available CPU cores
threads = str(os.cpu_count())
print(f"Using {threads} threads for IQ-TREE")

start_total = time.time()

for folder in species:
    print(f"={folder}=")
    iqtree_input_path = f"/content/hyphy/IQ_Tree_Input/{folder}"
    iqtree_out_dir = os.path.join(base_out, "IQ_Tree_tree", folder)
    hyphy_out_dir = os.path.join(base_out, "Hyphy_labeled_tree", folder)
    os.makedirs(iqtree_out_dir, exist_ok=True)
    os.makedirs(hyphy_out_dir, exist_ok=True)

    if folder == "Mammal":
        #if gene in ['NS1', 'M2', 'NP', 'NA', 'M1', 'NS2', 'PA', 'PB1', 'HA']:
          #print(f"\n=== Skipping {folder}/{gene} ===")
        print(f"\n=== Skipping {folder} === \n ==     ==")
        continue

    for fname in os.listdir(iqtree_input_path):
        if fname.endswith("_combined.fasta"):
            aln_path = os.path.join(iqtree_input_path, fname)
            gene = fname.replace("_combined.fasta", "")

            print(f"\n=== Processing {folder}/{gene} ===")

            start_gene = time.time()

            # Create per-gene output folders
            gene_iqtree_dir = os.path.join(iqtree_out_dir, gene)
            gene_hyphy_dir = os.path.join(hyphy_out_dir, gene)
            os.makedirs(gene_iqtree_dir, exist_ok=True)
            os.makedirs(gene_hyphy_dir, exist_ok=True)

            # === Step 1: Run IQ-TREE ===
            print("  → Running IQ-TREE")
            prefix = os.path.join(gene_iqtree_dir, gene)
            break

            subprocess.run([
                "iqtree2",
                "-s", aln_path,
                "-m", "GTR+I+G",
                "-T", "8",
                "-pre", prefix
            ], check=True)

        print("  → IQ-TREE done")

        # === Step 1.5: Copy the whole IQ_Tree_output folder to Drive ===
        drive_dest = f"/content/drive/MyDrive/Paper_Results/IQ_Tree_output/IQ_Tree_tree/{folder}"
        os.makedirs(drive_dest, exist_ok=True)
        break

        subprocess.run([
            "cp", "-r",
            gene_iqtree_dir,
            drive_dest
        ], check=True)

        print(f"  → Copied {gene_iqtree_dir} to {drive_dest}")

        treefile = prefix + ".treefile"
        labeled_tree = os.path.join(gene_hyphy_dir, gene + "_labeled.tree")

        # === Step 2: Run HyPhy LabelTree ===
        print("  → Running HyPhy LabelTree")
        try:
            subprocess.run([
                "hyphy", "res/TemplateBatchFiles/lib/label-tree.bf",
                "--tree", treefile,
                "--regexp", "2.3.3.4b",
                "--label", "2.3.3.4b",
                "--internal-nodes", "All descendants",
                "--leaf-nodes", "Label",
                "--output", labeled_tree
            ],
            check=True,
            text=True,
            capture_output=True
            )
            print(f"✅ {folder}/{gene}: labeled tree saved -> {labeled_tree}")
        except subprocess.CalledProcessError as e:
            print(f"❌ HyPhy failed for {folder}/{gene}")
            print("Return code:", e.returncode)
            print("Standard output:\n", e.stdout)
            print("Standard error:\n", e.stderr)

        # === Step 2.5: Copy this gene's HyPhy labeled results to Drive ===
        drive_dest_hyphy = f"/content/drive/MyDrive/Paper_Results/IQ_Tree_output/Hyphy_labeled_tree/{folder}"
        os.makedirs(drive_dest_hyphy, exist_ok=True)

        subprocess.run([
            "cp", "-r",
            gene_hyphy_dir,
            drive_dest_hyphy
        ], check=True)

        print(f"  → Copied {gene_hyphy_dir} to {drive_dest_hyphy}")

        elapsed_gene = time.time() - start_gene
        print(f"⏱ Runtime for {gene}: {elapsed_gene:.2f} seconds\n")

elapsed_total = time.time() - start_total
print(f"\n=== All jobs finished in {elapsed_total/60:.2f} minutes ===")




In [ ]:
#for the iqtree - to move from DRIVE to the GPU (for the NP run)
!cp -r "/content/drive/MyDrive/Paper_Results/IQ_Tree_output/IQ_Tree_tree/Avian/NP" "/content/hyphy/IQ_Tree_output/IQ_Tree_tree/Avian/"

#Running Hyphy ***Busted***

In [ ]:
%ls

cmake/                 contrib/              LICENSE
CMakeCache.txt         CONTRIBUTING.md       Makefile
CMakeFiles/            CTestTestfile.cmake   README.md
cmake_install.cmake    Dockerfile            res/
CMakeLists.txt         hyphy*                run_unit_tests.sh*
cmake_uninstall.cmake  HYPHYMPI*             src/
compile_commands.json  install_manifest.txt  tests/


In [ ]:
!hyphy busted --help


Available analysis command line options
---------------------------------------
Use --option VALUE syntax to invoke
If a [reqired] option is not provided on the command line, the analysis will prompt for its value
[conditionally required] options may or not be required based on the values of other options

code
	Which genetic code should be used
	default value: Universal

alignment [required]
	An in-frame codon alignment in one of the formats supported by HyPhy

tree [conditionally required]
	A phylogenetic tree (optionally annotated with {})
	applies to: Please select a tree file for the data:

branches
	Branches to test
	default value: All

srv
	Include synonymous rate variation in the model
	default value: Yes

rates
	The number omega rate classes to include in the model [1-10, default 3]
	default value: busted.rate_classes [computed at run time]

multiple-hits
	Include support for multiple nucleotide substitutions
	default value: None

mss
	Include support for multiple synonymous 

In [ ]:
#Move Hyphy_labeled_tree to cpu
!cp -r "/content/drive/MyDrive/Paper_Results/IQ_Tree_output/Hyphy_labeled_tree" "/content/hyphy/"

In [ ]:
#Move IQ_Tree_Input to cpu
!cp -r "/content/drive/MyDrive/Paper_Results/IQ_Tree_Input/" "/content/hyphy/"

In [ ]:
#To move the intermediate results to the cpu
#!cp -r "/content/drive/MyDrive/Paper_Results/Busted_results/" "/content/hyphy"

In [ ]:
#To move the alignment input from drive to cpu
!cp -r "/content/drive/MyDrive/Paper_Results/Busted_Alignment_Input/" "/content/hyphy"

In [ ]:
!mkdir -p "/content/hyphy/Busted_results/Avian"

In [ ]:
from Bio import SeqIO
#Genes finished:
  # M1, M2, NA, NP, NS1, NS2, PB1
#Running: HA,  PB2
#Remaining PA
fasta_in = "/content/hyphy/IQ_Tree_Input/Avian/PB2_combined.fasta"
fasta_out = "/content/hyphy/Busted_Alignment_Input/Avian/PB2_combined.fasta"

count_replaced = 0

with open(fasta_out, "w") as out:
    for record in SeqIO.parse(fasta_in, "fasta"):
        original_id = record.id
        # check for unwanted characters
        if any(char in record.id for char in [",", "(", ")", "'", '"']):
            record.id = (
                record.id
                .replace(",", "_")
                .replace("(", "_")
                .replace(")", "_")
                .replace("'", "_")
                .replace('"', "_")
            )
            count_replaced += 1
        record.description = ""
        SeqIO.write(record, out, "fasta")

print(f"✅ Done. {count_replaced} record IDs contained commas or parentheses and were replaced.")


✅ Done. 20 record IDs contained commas or parentheses and were replaced.


In [ ]:
!cp -r "/content/drive/MyDrive/Paper_Results/Busted_results/Avian/PB2.json" "/content/hyphy/Busted_results/Avian/"
!cp -r "/content/drive/MyDrive/Paper_Results/Busted_results/Avian/PB2_intermediate.json" "/content/hyphy/Busted_results/Avian/"

In [ ]:

#For intermediate fits
!hyphy busted \
  --alignment "/content/hyphy/Busted_Alignment_Input/Avian/PB2_combined.fasta" \
  --tree "/content/hyphy/Hyphy_labeled_tree/Avian/PB2/PB2_labeled.tree" \
  --branches "2.3.3.4b" \
  --output "/content/hyphy/Busted_results/Avian/PB2.json" \
  --kill-zero-lengths Yes \
  --intermediate-fits "/content/hyphy/Busted_results/Avian/PB2_intermediate.json"


#To move the alignment input from cpu to drive
!cp -r "/content/hyphy/Busted_Alignment_Input/Avian" "/content/drive/MyDrive/Paper_Results/Busted_Alignment_Input/"

#To move intermediate json from cpu to drive
!cp "/content/hyphy/Busted_results/Avian/PB2_intermediate.json" "/content/drive/MyDrive/Paper_Results/Busted_results/Avian/"

#To move final json output from cpu to drive
!cp "/content/hyphy/Busted_results/Avian/PB2.json" "/content/drive/MyDrive/Paper_Results/Busted_results/Avian/"


Analysis Description
--------------------
BUSTED (branch-site unrestricted statistical test of episodic
diversification) uses a random effects branch-site model fitted jointly
initial search phase to use adaptive Nedler-Mead. Version 3.0 implements
the option for branch-site variation in synonymous substitution rates.
Version 3.1 adds HMM auto-correlation option for SRV, and binds SRV
distributions for multiple branch sets. Version 4.0 adds support for
multiple hits (MH), ancestral state reconstruction saved to JSON, and
profiling of branch-site level support for selection / multiple hits.
Version 4.2 adds calculation of MH-attributable fractions of
substitutions. Version 4.5 adds an 'error absorption' component Version
4.6 adds support for MSS-type models 
to all or a subset of tree branches in order to test for alignment-wide
evidence of episodic diversifying selection. Assuming there is evidence
of positive selection (i.e. there is an omega > 1), BUSTED will also
perform a quick ev

In [ ]:
#To move the alignment input from cpu to drive
!cp -r "/content/hyphy/Busted_Alignment_Input/Avian" "/content/drive/MyDrive/Paper_Results/Busted_Alignment_Input/"

#To move intermediate json from cpu to drive
!cp "/content/hyphy/Busted_results/Avian/PB2_intermediate.json" "/content/drive/MyDrive/Paper_Results/Busted_results/Avian/"

#To move final json output from cpu to drive
!cp "/content/hyphy/Busted_results/Avian/PB2.json" "/content/drive/MyDrive/Paper_Results/Busted_results/Avian/"

In [ ]:
#Script to automate the busted run
import os, subprocess

input_dir = "/content/hyphy/IQ_Tree_Input/Avian"
tree_dir = "/content/hyphy/IQ_Tree_output/Hyphy_labeled_tree/Avian"
output_dir = "/content/hyphy/BUSTEDS_results/Avian"
os.makedirs(output_dir, exist_ok=True)

for fname in os.listdir(input_dir):
    if fname.endswith("_combined.fasta"):
        gene = fname.replace("_combined.fasta", "")
        aln = os.path.join(input_dir, fname)
        tree = os.path.join(tree_dir, gene, f"{gene}_labeled.tree")
        out = os.path.join(output_dir, f"{gene}_BUSTEDS.json")

        subprocess.run([
            "hyphy", "busted-s",
            "--alignment", aln,
            "--tree", tree,
            "--branches", "2.3.3.4b",
            "--srv", "Yes",
            "--output", out
        ], check=True)


#Running Hyphy ***MEME***

In [ ]:
!hyphy meme --help
#To visualize meme
#https://observablehq.com/@spond?type=collections


Available analysis command line options
---------------------------------------
Use --option VALUE syntax to invoke
If a [reqired] option is not provided on the command line, the analysis will prompt for its value
[conditionally required] options may or not be required based on the values of other options

code
	Which genetic code should be used
	default value: Universal

alignment [required]
	An in-frame codon alignment in one of the formats supported by HyPhy

tree [conditionally required]
	A phylogenetic tree (optionally annotated with {})
	applies to: Please select a tree file for the data:

branches
	Branches to test
	default value: All

pvalue
	The p-value threshold to use when testing for selection
	default value: 0.1

resample
	[Advanced setting, will result in MUCH SLOWER run time] Perform parametric bootstrap resampling to derive site-level null LRT distributions up to this many replicates per site. Recommended use for small to medium (<30 sequences) datasets
	default value:

In [ ]:
#Move Hyphy_labeled_tree to cpu
!cp -r "/content/drive/MyDrive/Paper_Results/IQ_Tree_output/Hyphy_labeled_tree" "/content/hyphy/"
#Move IQ_Tree_Input to cpu
!cp -r "/content/drive/MyDrive/Paper_Results/IQ_Tree_Input/" "/content/hyphy/"
#To move the alignment input from drive to cpu
!cp -r "/content/drive/MyDrive/Paper_Results/Busted_Alignment_Input/" "/content/hyphy"
#To move the MEME_results results from drive to cpu
!cp -r "/content/drive/MyDrive/Paper_Results/MEME_results/" "/content/hyphy/"

In [ ]:
"""
!hyphy meme \
  --alignment "/content/hyphy/Busted_Alignment_Input/Mammal/NP_combined.fasta" \
  --tree "/content/hyphy/Hyphy_labeled_tree/Mammal/NP/NP_labeled.tree" \
  --branches "2.3.3.4b" \
  --pvalue 0.05 \
  --rates 2 \
  --output "/content/hyphy/MEME_results/Mammal/NP.json"
"""


Analysis Description
--------------------
MEME (Mixed Effects Model of Evolution) estimates a site-wise synonymous
(&alpha;) and a two-category mixture of non-synonymous (&beta;-, with
Multiple partitions within a NEXUS file are also supported for
recombination - aware analysis. Version 3.0 adds a different format for
ancestral state reconstruction, branch-site posterior storage, and
site-level heterogeneity testing. Version 4 adds support for multiple
hits and more than 2 rate classes on omega, as well as site-level
imputation option. Version 4.1 fixes bugs with EBF computation for > 2
more rates and MH reporting. 
proportion p-, and &beta;+ with proportion [1-p-]) rates, and uses a
likelihood ratio test to determine if &beta;+ > &alpha; at a site. The
estimates aggregate information over a proportion of branches at a site,
so the signal is derived from episodic diversification, which is a
combination of strength of selection [effect size] and the proportion of
the tree affected. A s

In [ ]:
import os
import subprocess
import shutil

# === Paths ===
species = "Mammal"
alignment_dir = f"/content/hyphy/Busted_Alignment_Input/{species}"
tree_base_dir = f"/content/hyphy/Hyphy_labeled_tree/{species}"
output_base_dir = f"/content/hyphy/MEME_results/{species}"
drive_base_dir = f"/content/drive/MyDrive/Paper_Results/MEME_results/{species}"

# Ensure output directories exist
#os.makedirs(output_base_dir, exist_ok=True)
#os.makedirs(drive_base_dir, exist_ok=True)

# Loop through all alignment files
for file in os.listdir(alignment_dir):
    if not file.endswith(".fasta"):
        continue

    gene_name = file.replace("_combined.fasta", "")
    alignment_path = os.path.join(alignment_dir, file)
    tree_path = os.path.join(tree_base_dir, gene_name, f"{gene_name}_labeled.tree")
    output_json = os.path.join(output_base_dir, f"{gene_name}.json")
    intermediate_json = os.path.join(output_base_dir, f"{gene_name}_intermediate.json")

    # Skip if tree does not exist
    if not os.path.exists(tree_path):
        print(f"⚠️ Skipping {gene_name} — tree not found")
        continue

    print(f"\n=== Running MEME for {gene_name} ===")

    # Run MEME
    cmd = [
        "hyphy", "meme",
        "--alignment", alignment_path,
        "--tree", tree_path,
        "--branches", "2.3.3.4b",
        "--pvalue", "0.05",
        "--rates", "2",
        "--output", output_json,
        "--kill-zero-lengths", "Yes",
        "--intermediate-fits", intermediate_json
    ]

    try:
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)
        print(result.stdout)
        print(f"✅ MEME completed for {gene_name}")

        # Copy JSON to Google Drive
        shutil.copy2(output_json, drive_base_dir)
        print(f"📁 Copied {gene_name}.json to Drive successfully.\n")

    except subprocess.CalledProcessError as e:
        print(f"❌ Error running MEME for {gene_name}")
        print("---- STDERR ----")
        print(e.stderr)
        print("---- STDOUT ----")
        print(e.stdout)
    except Exception as e:
        print(f"⚠️ Copy error for {gene_name}: {e}")


⚠️ Skipping M1 — Completed
⚠️ Skipping M2 — Completed

=== Running MEME for PA ===

Analysis Description
--------------------
MEME (Mixed Effects Model of Evolution) estimates a site-wise synonymous
(&alpha;) and a two-category mixture of non-synonymous (&beta;-, with
Multiple partitions within a NEXUS file are also supported for
recombination - aware analysis. Version 3.0 adds a different format for
ancestral state reconstruction, branch-site posterior storage, and
site-level heterogeneity testing. Version 4 adds support for multiple
hits and more than 2 rate classes on omega, as well as site-level
imputation option. Version 4.1 fixes bugs with EBF computation for > 2
more rates and MH reporting. 
proportion p-, and &beta;+ with proportion [1-p-]) rates, and uses a
likelihood ratio test to determine if &beta;+ > &alpha; at a site. The
estimates aggregate information over a proportion of branches at a site,
so the signal is derived from episodic diversification, which is a
combination 

In [ ]:
!cp -r "/content/hyphy/MEME_results/" "/content/drive/MyDrive/Paper_Results/"

#Running Hyphy ***FEL***

In [ ]:
!hyphy FEL --help


Available analysis command line options
---------------------------------------
Use --option VALUE syntax to invoke
If a [reqired] option is not provided on the command line, the analysis will prompt for its value
[conditionally required] options may or not be required based on the values of other options

code
	Which genetic code should be used
	default value: Universal

alignment [required]
	An in-frame codon alignment in one of the formats supported by HyPhy

tree [conditionally required]
	A phylogenetic tree (optionally annotated with {})
	applies to: Please select a tree file for the data:

branches
	Branches to test. Options: 'All' (default), 'Internal', 'Leaves', 'Unlabeled', 'fg' (labeled branches), comma-separated branch names (e.g. 'Node1,Node2'), or regex patterns (e.g. '/^human/i')
	default value: All

srv
	Include synonymous rate variation in the model
	default value: Yes

multiple-hits
	Include support for multiple nucleotide substitutions
	default value: None

pvalue
	T

In [ ]:
#Move Hyphy_labeled_tree to cpu
!cp -r "/content/drive/MyDrive/Paper_Results/IQ_Tree_output/Hyphy_labeled_tree" "/content/hyphy/"
#Move IQ_Tree_Input to cpu
!cp -r "/content/drive/MyDrive/Paper_Results/IQ_Tree_Input/" "/content/hyphy/"
#To move the alignment input from drive to cpu
!cp -r "/content/drive/MyDrive/Paper_Results/Busted_Alignment_Input/" "/content/hyphy"
#Make output FEL_results directory
!mkdir -p "/content/hyphy/FEL_results/Mammal"


In [ ]:
"""
!hyphy FEL \
  --alignment "/content/hyphy/Busted_Alignment_Input/Mammal/PB2_combined.fasta" \
  --tree "/content/hyphy/Hyphy_labeled_tree/Mammal/PB2/PB2_labeled.tree" \
  --branches "2.3.3.4b" \
  --output "/content/hyphy/FEL_results/Mammal/PB2.json" \
  --kill-zero-lengths Yes \
  --intermediate-fits "/content/hyphy/FEL_results/Mammal/PB2_intermediate.json"
"""


Analysis Description
--------------------
FEL (Fixed Effects Likelihood) estimates site-wise synonymous (&alpha;)
and non-synonymous (&beta;) rates, and uses a likelihood ratio test to
determine if beta &neq; alpha at a site. The estimates aggregate
information over all branches, so the signal is derived from pervasive
diversification or conservation. A subset of branches can be selected
for testing as well, in which case an additional (nuisance) parameter
will be inferred -- the non-synonymous rate on branches NOT selected for
testing. Multiple partitions within a NEXUS file are also supported for
recombination - aware analysis. Version 2.5 adds MH support. Version 2.6
adds substitution mapping. 

- __Requirements__: in-frame codon alignment and a phylogenetic tree

- __Citation__: Not So Different After All: A Comparison of Methods for Detecting Amino
Acid Sites Under Selection (2005). _Mol Biol Evol_ 22 (5): 1208-1222

- __Written by__: Sergei L Kosakovsky Pond and Simon DW Frost



In [ ]:
import os
import subprocess
import shutil

# === Paths ===
species = "Mammal"
alignment_dir = "/content/hyphy/Busted_Alignment_Input/"+species
tree_base_dir = "/content/hyphy/Hyphy_labeled_tree/"+species
output_base_dir = "/content/hyphy/FEL_results/"+species
drive_base_dir = "/content/drive/MyDrive/Paper_Results/FEL_results/"+species

# === Ensure output directories exist ===
os.makedirs(output_base_dir, exist_ok=True)
os.makedirs(drive_base_dir, exist_ok=True)

# === Loop through all FASTA alignment files ===
for file in os.listdir(alignment_dir):
    if not file.endswith(".fasta"):
        continue

    gene_name = file.replace("_combined.fasta", "")
    alignment_path = os.path.join(alignment_dir, file)
    tree_path = os.path.join(tree_base_dir, gene_name, f"{gene_name}_labeled.tree")
    output_json = os.path.join(output_base_dir, f"{gene_name}.json")
    intermediate_json = os.path.join(output_base_dir, f"{gene_name}_intermediate.json")

    # === Check if tree exists ===
    if not os.path.exists(tree_path):
        print(f"⚠️ Skipping {gene_name} — tree not found")
        continue

    if gene_name != 'M1':
      continue

    if gene_name in ['HA']:
      print(f"\n=== Skipping {species}/{gene_name} - Completed ===")
      continue

    print(f"\n=== Processing {species}/{gene_name} ===")

    # === Run HyPhy FEL ===
    cmd = [
        "hyphy", "FEL",
        "--alignment", alignment_path,
        "--tree", tree_path,
        "--branches", "2.3.3.4b",
        "--output", output_json,
        "--pvalue", "0.05",
        "--kill-zero-lengths", "Yes",
        "--intermediate-fits", intermediate_json
    ]

    try:
        subprocess.run(cmd, check=True)
        print(f"✅ Completed FEL for {gene_name}")

        # === Copy results to Google Drive ===
        shutil.copy2(intermediate_json, drive_base_dir)
        shutil.copy2(output_json, drive_base_dir)
        print(f"📁 Moved {gene_name} results to Drive successfully.\n")

    except subprocess.CalledProcessError:
        print(f"❌ Error running FEL for {species}/{gene_name}")
    except Exception as e:
        print(f"⚠️ Copy error for {species}/{gene_name}: {e}")



=== Processing Mammal/M1 ===
✅ Completed FEL for M1
📁 Moved M1 results to Drive successfully.



In [ ]:
#To move the alignment input from cpu to drive
#!cp -r "/content/hyphy/Busted_Alignment_Input/Avian" "/content/drive/MyDrive/Paper_Results/Busted_Alignment_Input/"

#To move intermediate json from cpu to drive
!cp "/content/hyphy/FEL_results/Mammal/HA_intermediate.json" "/content/drive/MyDrive/Paper_Results/FEL_results/Mammal/"

#To move final json output from cpu to drive
!cp "/content/hyphy/FEL_results/Mammal/HA.json" "/content/drive/MyDrive/Paper_Results/FEL_results/Mammal/"

#Running Hyphy ***RELAX***

In [ ]:
!hyphy relax --help
#To visualize meme
#https:/observablehq.com/@spond?type=collections


Available analysis command line options
---------------------------------------
Use --option VALUE syntax to invoke
If a [reqired] option is not provided on the command line, the analysis will prompt for its value
[conditionally required] options may or not be required based on the values of other options

multiple-files
	Use multiple files as input
	default value: No

filelist [required]
	A line list of file paths for the alignments to include in this analysis

code
	Which genetic code should be used
	default value: Universal

code
	Which genetic code should be used
	default value: Universal

alignment [required]
	An in-frame codon alignment in one of the formats supported by HyPhy

tree [conditionally required]
	A phylogenetic tree (optionally annotated with {})
	applies to: Please select a tree file for the data:

mode
	Run mode
	default value: Classic mode
	applies to: Group test mode

test [conditionally required]
	Branches to use as the test set
	applies to: Choose the set of br

In [ ]:
#Move Hyphy_labeled_tree to cpu
!cp -r "/content/drive/MyDrive/Paper_Results/IQ_Tree_output/Hyphy_labeled_tree" "/content/hyphy/"
#Move IQ_Tree_Input to cpu
!cp -r "/content/drive/MyDrive/Paper_Results/IQ_Tree_Input/" "/content/hyphy/"
#To move the alignment input from drive to cpu
!cp -r "/content/drive/MyDrive/Paper_Results/Busted_Alignment_Input/" "/content/hyphy"
#To move the Relax_results folder from drive to cpu
#!cp -r "/content/drive/MyDrive/Paper_Results/Relax_results/" "/content/hyphy/"

To find relax results: pval and K - Search for "LRT" in the json file

In [ ]:
import os
import subprocess
import shutil

# === Paths ===
species = "Mammal"
alignment_dir = f"/content/hyphy/Busted_Alignment_Input/{species}"
tree_base_dir = f"/content/hyphy/Hyphy_labeled_tree/{species}"
output_base_dir = f"/content/hyphy/Relax_results_2/{species}"
drive_base_dir = f"/content/drive/MyDrive/Paper_Results/Relax_results_2/{species}"

# Ensure output directories exist
#os.makedirs(output_base_dir, exist_ok=True)
#os.makedirs(drive_base_dir, exist_ok=True)

# Loop through all alignment files
for file in os.listdir(alignment_dir):
    if not file.endswith(".fasta"):
        continue

    gene_name = file.replace("_combined.fasta", "")
    alignment_path = os.path.join(alignment_dir, file)
    tree_path = os.path.join(tree_base_dir, gene_name, f"{gene_name}_labeled.tree")
    output_base_dir = f"/content/hyphy/Relax_results_2/{species}"
    output_json = os.path.join(output_base_dir, f"{gene_name}.json")
    intermediate_json = os.path.join(output_base_dir, f"{gene_name}_intermediate.json")

    # Skip if tree does not exist
    if not os.path.exists(tree_path):
        print(f"⚠️ Skipping {gene_name} — tree not found")
        continue

    print(f"\n=== Running RELAX for {gene_name} ===")

    # Run Relax
    cmd = [
        "hyphy", "relax",
        "--alignment", alignment_path,
        "--tree", tree_path,
        "--test", "2.3.3.4b",
        "--output", output_json,
        "--intermediate-fits", intermediate_json
    ]

    try:
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)
        print(result.stdout)
        print(f"✅ Relax completed for {gene_name}")

        # Copy JSON to Google Drive
        #shutil.copy2(output_json, drive_base_dir)
        print(f"📁 Copied {gene_name}.json to Drive successfully.\n")

    except subprocess.CalledProcessError as e:
        print(f"❌ Error running Relax for {gene_name}")
        print("---- STDERR ----")
        print(e.stderr)
        print("---- STDOUT ----")
        print(e.stdout)
    except Exception as e:
        print(f"⚠️ Copy error for {gene_name}: {e}")



=== Running RELAX for PA ===
multiple-files: No

Analysis Description
--------------------
RELAX (a random effects test of selection relaxation) uses a random
effects branch-site model framework to test whether a set of 'Test'
for convergence diagnostics. Version 4.5 adds support for multiple
datasets for joint testing.
branches evolves under relaxed selection relative to a set of
'Reference' branches (R), as measured by the relaxation parameter (K).
Version 2.1 adds a check for stability in K estimates to try to mitigate
convergence problems. Version 3 provides support for >2 branch sets.
Version 3.1 adds LHC + Nedler-Mead initial fit phase and keyword
support. Version 3.1.1 adds some bug fixes for better convergence.
Version 4.0 adds support for synonymous rate variation. Version 4.1 adds
further support for multiple hit models. Version 4.1.1 adds reporting

- __Requirements__: in-frame codon alignment and a phylogenetic tree, with at least two
groups of branches defined using the {

In [ ]:
#To move the Relax_results folder from cpu to drive
!cp -r "/content/hyphy/Relax_results/" "/content/drive/MyDrive/Paper_Results/"